### Read Smallcases

In [1]:
import re
import numpy as np
import pandas as pd
import requests

In [2]:
# Connect to services
import os
import sys

backend_path = os.path.abspath(os.path.join(os.getcwd(), '../backend'))
sys.path.append(backend_path)

from app.internal.firebase import db, bucket

In [3]:
ids = [
    {"id": "SCAW_0001", "name": "All Weather Investing"},
    {"id": "SCMO_0015", "name": "Safe Haven"},
    {"id": "SCMO_0026", "name": "Growth & Income"},
    {"id": "SCMO_0016", "name": "The Naked Trader"},
    {"id": "SCMO_0024", "name": "The PE List"},
    {"id": "SCMO_0013", "name": "Dividend Stars"},
    {"id": "SCMO_0014", "name": "Dividend Aristocrats"},
    {"id": "SCNM_0025", "name": "Electric Mobility"},
    {"id": "SCNM_0012", "name": "Rising Rural Demand"},
    {"id": "SCTR_0014", "name": "House of Tata"},
    {"id": "SCTR_0017", "name": "House of Murugappa"},
    {"id": "SCTR_0010", "name": "Realty Tracker"},
]
smallcase_id = ids[0]["id"]
smallcase_sheet_path = f"data/smallcases/{smallcase_id}_Timeline.xlsx"
smallcase_sheet = pd.ExcelFile(smallcase_sheet_path)

constituents = pd.read_excel(smallcase_sheet, sheet_name="Historical Constituents")
constituents.head()

,Date Range,Constituents,Weightage
0,2018-07-16 to 2018-07-16,Nippon India ETF Nifty 50 BeES,0.19
1,NaN,Nippon India ETF Nifty Next 50 Junior BeES,0.14
2,NaN,Nippon India ETF Gold BeES,0.41
3,NaN,Nippon India ETF Nifty 1D Rate Liquid BeES,0.25
4,2018-07-17 to 2018-09-27,Nippon India ETF Nifty 50 BeES,0.24


In [4]:
indexes = pd.read_excel(smallcase_sheet, sheet_name="Historical Index Values")
indexes.head()

,Date,All Weather Investing,NIFTY 100,Rebalance Occured
0,2018-07-17,100.00,100.00,True
1,2018-07-18,99.33,99.63,NaN
2,2018-07-19,99.29,99.37,NaN
3,2018-07-20,99.52,99.87,NaN
4,2018-07-23,100.22,100.68,NaN


### Fetch and POST Smallcase and Stats

In [5]:
smallcase_base_url = "https://api.smallcase.com"
resp = requests.get(f"{smallcase_base_url}/smallcases/smallcase?scid={smallcase_id}")

resp.status_code

200

In [6]:
local_base_url = "http://localhost:7999"
smallcase = resp.json()["data"]
stats = smallcase["stats"]

def remove_p_tag(text: str):
    return re.sub(r"</?(p|ul|li)>", "", text)

methodologies = []
for m in smallcase["methodology"]:
    methodologies.append({
        "key": m["key"],
        "details": remove_p_tag(m["content"]),
    })

investment_strategies = []
for i in smallcase["info"]["investmentStrategy"]:
    investment_strategies.append(i["key"])

payload = {
    "id": smallcase_id,
    "name": smallcase["info"]["name"],
    "slug": smallcase["info"]["slug"],
    "description": smallcase["info"]["shortDescription"],
    "volatility": stats["ratios"]["riskLabel"],
    "popularity_rank": smallcase["flags"]["popular"]["rank"],
    "contains_etf": smallcase["flags"]["containsEtf"],
    "contains_stock": smallcase["flags"]["containsStock"],
    "constituent_count": smallcase["constituentsCount"],
    "growth_since_launch": {
        "cagr": stats["ratios"]["cagr"],
        "returns": stats["returns"]["sinceInception"],
        "duration": stats["ratios"]["cagrDuration"],
    },
    "benchmark": {
        "id": smallcase["benchmark"]["id"],
        "index": smallcase["benchmark"]["index"],
        "details": smallcase["benchmark"]["msg"],
    },
    "methodologies": methodologies,
    "launch_date": smallcase["info"]["uploaded"],
    "inception_date": smallcase["info"]["created"],
    "last_rebalance_date": smallcase["info"]["lastRebalanced"],
    "next_rebalance_date": smallcase["info"]["nextUpdate"],
    "investment_strategies": investment_strategies,
    "rebalance_frequency": smallcase["info"]["rebalanceSchedule"],
}

local_resp = requests.post(f"{local_base_url}/smallcases/", json=payload)
local_resp.status_code


201

In [8]:
# Date of scraping
year = "2022"
month = "05"

payload = {
    "year": year,
    "month": month,
    "min_sip_amount": stats["minSipAmount"],
    "investor_count": stats["investorCount"],
    "subscriber_count": stats["subscriberCount"],
    
    "cagr": {
        "one_year": stats["ratios"]["cagr1y"],
        "three_year": stats["ratios"]["cagr3y"],
        "five_year": stats["ratios"]["cagr5y"],
    },
    "returns": {
        "monthly": stats["returns"]["monthly"],
        "quarterly": stats["returns"]["quarterly"],
        "half_year": stats["returns"]["halfyearly"],
        "one_year": stats["returns"]["yearly"],
        "three_year": stats["returns"]["threeYear"],
        "five_year": stats["returns"]["fiveYear"],
    },
    "ratios": {
        "dividend_yield": stats["ratios"]["divYield"],
        "dividend_yield_differential": stats["ratios"]["divYieldDifferential"],

        "risk": stats["ratios"]["risk"],
        "pe": stats["ratios"]["pe"],
        "pb": stats["ratios"]["pb"],
        "beta": stats["ratios"]["beta"],
        "sharpe": stats["ratios"]["sharpeRatio"],
    },
    "weightage": {
        "large_cap": stats["ratios"]["largeCapPercentage"],
        "mid_cap": stats["ratios"]["midCapPercentage"],
        "small_cap": stats["ratios"]["smallCapPercentage"],
        "market_category": stats["ratios"]["marketCapCategory"]
    }
}

local_resp = requests.post(f"{local_base_url}/smallcases/{smallcase_id}/statistics/", json=payload)
local_resp.status_code

201

### Upload XLSX to bucket

In [ ]:
blob = bucket.blob(f"smallcases/{smallcase_id}")
blob.upload_from_filename(smallcase_sheet_path)


### Map and POST Smallcase Name to Upstox Instruments

In [ ]:
postable_constituents = constituents["Constituents"].unique()
len(postable_constituents)

In [ ]:
successful_constitents = []
failed_constitents = []

async def query_ticker_name(name: str):
    return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()

for c in postable_constituents:
    q = c.upper()
    docs = await query_ticker_name(q)
    if len(docs) == 0:
        failed_constitents.append(c)
    else:
        for e in docs:
            await db.collection("tickers").document(e.id).update({
                "smallcase_name": c
            })
            successful_constitents.append((c, e.id))

In [ ]:
len(successful_constitents), len(failed_constitents)

In [ ]:
# # All return upper

def replace_ltd(s):
    return s.upper().replace("LTD", "LIMITED")

# def replace_technologies(s):
#     return s.upper().replace("TECHNOLOGIES", "TECHNO.")

# def replace_company(s):
#     return s.upper().replace("COMPANY", "CO")

# def replace_corporation(s):
#     return s.upper().replace("CORPORATION", "CORP")

# Ltd -> Limited
for f in failed_constitents:
    q = replace_ltd(f)
    docs = await query_ticker_name(q)
    if len(docs) != 0:
        for e in docs:
            await db.collection("tickers").document(e.id).update({
                "smallcase_name": f
            })
            successful_constitents.append((f, e.id))
            failed_constitents.remove(f)

In [ ]:
len(successful_constitents), len(failed_constitents)

In [ ]:
nse = pd.read_csv("data/upstox/NSE.csv")
equities = nse.query('instrument_type == "EQUITY" and exchange == "NSE_EQ"')
equities.head()

In [ ]:
import difflib
df = equities.filter(['name', "exchange_token"])
tickers = pd.DataFrame() 
r = {}
for f in failed_constitents:
    df['similarity'] = df["name"].apply(lambda x: difflib.SequenceMatcher(None, x.lower(), f.lower()).ratio())
        
    top_match = df.sort_values(by='similarity', ascending=False).head(1)
    top_match['smallcase_name'] = f
        
    tickers = pd.concat([tickers, top_match], ignore_index=True)

tickers

In [ ]:
payload = tickers[['exchange_token', 'smallcase_name']].to_dict(orient='records')
for p in payload:
    id = str(int(p['exchange_token']))
    await db.document(f"tickers/{id}").update({"smallcase_name": p["smallcase_name"]})
    successful_constitents.append((p["smallcase_name"], id))

In [ ]:
len(successful_constitents), len(successful_constitents) == len(postable_constituents)